# Final Report
By: Jenny Zhang, Marcus Kang, Joshua Chang, Daniel Lee

# Introduction

# Data Description

# Pre-registered Analyses

## Hypothesis 1: 
Yards per Reception and Yards after the Catch are most indicative of good post-season rank for a wide receiver.

## Hypothesis 2:
The number of points scored by wide receivers has increased over time.

# Data Analysis

## Data Cleaning

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
wr_df = pd.read_csv("wr_df.csv")

## Summary Statistics

### Post Season Rank

### Scoring Points Over Time

# Models

## Model (1)

## Model (2)

# Evaluation of Significance

## Model (1)

## Model (2)

# Interpretations and Conclusions

## Model (1)
## Model (2)

## Conclusions

# Limitations

## General

## Model (1)

## Model (2)